In [ ]:
%pip install statsmodels numpy

In [1]:
# 临床样本量估算
import numpy as np
from statsmodels.stats.power import GofChisquarePower

def sample_size_three_proportions(p, alpha=0.05, power=0.8):
    """
    计算三组比例比较（卡方检验）所需样本量（每组相等）
    p: list or array of proportions
    """
    p = np.array(p)

    # 总体比例
    p0 = np.mean(p)

    # 卡方效应量 Cohen's W = sqrt(Σ((p_i - p0)^2 / p0))
    w = np.sqrt(np.sum((p - p0)**2 / p0))

    power_analysis = GofChisquarePower()

    # 求样本量：chi-square goodness-of-fit test
    n_total = power_analysis.solve_power(
        effect_size=w,
        nobs=None,
        alpha=alpha,
        power=power
    )

    # 三组等分
    n_per_group = np.ceil(n_total / len(p))

    return n_per_group, w


# 示例使用
p = [0.6, 0.84, 0.95]       # 三组有效率
alpha = 0.05
power = 0.8

n, w = sample_size_three_proportions(p, alpha, power)

print("Cohen's W effect size:", w)
print("每组所需样本量:", n)
print("三组总样本量:", n * 3)


Cohen's W effect size: 0.2835813993227359
每组所需样本量: 33.0
三组总样本量: 99.0


In [6]:
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

# 两组有效率
p1 = 0.52   # 组1有效率
p2 = 0.80   # 组2有效率

# 显著性水平
alpha = 0.05

# power=0.8（80% 抗拒第二类错误的概率）
power = 0.8

# 计算效应量
effect_size = proportion_effectsize(p1, p2)
print("Effect size =", effect_size)

# 计算所需样本量（每组）
analysis = NormalIndPower()
sample_size_per_group = analysis.solve_power(effect_size=effect_size,
                                             alpha=alpha,
                                             power=power,
                                             ratio=1)  # 1 表示等样本量

print("每组所需样本量 ≈", sample_size_per_group)


Effect size = -0.6034904344392951
每组所需样本量 ≈ 43.1018414465025


In [2]:
import math

def calculate_sample_size(p_max, p_min, lambda_val):
    """
    根据完全随机设计多个率比较的公式计算每组样本量 n。
    
    公式: n = 2λ / (2sin⁻¹√p_max - 2sin⁻¹√p_min)²
    
    参数:
    ----------
    p_max : float
        预期的最大率 (例如 85.5% 输入 0.855)
    p_min : float
        预期的最小率 (例如 75.5% 输入 0.755)
    lambda_val : float
        查表得到的 λ 值 (取决于 α, β 和自由度 v=k-1)
        
    返回:
    -------
    n : float
        计算出的每组所需样本量 (通常需要向上取整)
    """
    
    # 1. 计算反正弦平方根变换 (结果为弧度)
    # 2*asin(sqrt(p)) 是方差稳定化变换
    trans_max = 2 * math.asin(math.sqrt(p_max))
    trans_min = 2 * math.asin(math.sqrt(p_min))
    
    # 2. 计算分母：(最大值变换值 - 最小值变换值) 的平方
    diff_sq = (trans_max - trans_min) ** 2
    
    # 防止分母为0 (即最大率等于最小率的情况)
    if diff_sq == 0:
        print("错误：最大率和最小率不能相同。")
        return None
        
    # 3. 计算样本量 n
    n = (2 * lambda_val) / diff_sq
    
    return n

# ==========================================
# 示例：复现图片中的【例 13-9】
# ==========================================

# 输入已知条件
P_MAX = 0.95   # 甲复方 95%
P_MIN = 0.6   # 丙复方 60%
LAMBDA = 12.65  # 查表值 (alpha=0.05, beta=0.10, v=2)

# 调用函数
n_calc = calculate_sample_size(P_MAX, P_MIN, LAMBDA)

if n_calc:
    print("-" * 30)
    print(f"输入参数: p_max={P_MAX}, p_min={P_MIN}, λ={LAMBDA}")
    print(f"公式计算结果: {n_calc:.2f}")  # 保留两位小数
    print(f"最终建议样本量 (每组): {math.ceil(n_calc)}") # 向上取整
    print("-" * 30)

# 验证：代码输出应为 391.18，与书中一致。

------------------------------
输入参数: p_max=0.95, p_min=0.6, λ=12.65
公式计算结果: 29.99
最终建议样本量 (每组): 30
------------------------------
